In [13]:
import pyodbc
server = 'act4-server.database.windows.net'
database = 'act4-database'
username = 'stanford'
password = 'Hola1234'

# Define the connection string for SQL Server (Azure)
conn_str = (
    f'DRIVER={{ODBC Driver 18 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)

# Establish the connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connected to the Azure SQL Database successfully!")

except pyodbc.Error as e:
    print(f"Error connecting to the database: {e}")

Connected to the Azure SQL Database successfully!


In [14]:
import pandas as pd

query = "SELECT * FROM SalesLT.Customer"
df = pd.read_sql(query, conn)

print(df.head())

C:\Users\david\AppData\Local\Temp\ipykernel_20528\2608267317.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   CustomerID  NameStyle Title FirstName MiddleName    LastName Suffix  \
0           1      False   Mr.   Orlando         N.         Gee   None   
1           2      False   Mr.     Keith       None      Harris   None   
2           3      False   Ms.     Donna         F.    Carreras   None   
3           4      False   Ms.     Janet         M.       Gates   None   
4           5      False   Mr.      Lucy       None  Harrington   None   

                  CompanyName               SalesPerson  \
0                A Bike Store   adventure-works\pamela0   
1          Progressive Sports    adventure-works\david8   
2    Advanced Bike Components  adventure-works\jillian0   
3       Modular Cycle Systems  adventure-works\jillian0   
4  Metropolitan Sports Supply      adventure-works\shu0   

                   EmailAddress         Phone  \
0  orlando0@adventure-works.com  245-555-0173   
1    keith0@adventure-works.com  170-555-0127   
2    donna0@adventure-works.com  279-555-0130   
3   

In [15]:
df.dtypes

CustomerID               int64
NameStyle                 bool
Title                   object
FirstName               object
MiddleName              object
LastName                object
Suffix                  object
CompanyName             object
SalesPerson             object
EmailAddress            object
Phone                   object
PasswordHash            object
PasswordSalt            object
rowguid                 object
ModifiedDate    datetime64[ns]
dtype: object

In [16]:
df['ModifiedDate'] = pd.to_datetime(df['ModifiedDate'])

min_date = df['ModifiedDate'].min()
df['ModifiedDateDays'] = (df['ModifiedDate'] - min_date).dt.days



In [17]:
useful_cols = ['CustomerID', 'CompanyName', 'SalesPerson', 'ModifiedDateDays']
df_clean = df[useful_cols].copy()

df_clean = pd.get_dummies(df_clean, columns=['CompanyName', 'SalesPerson'], drop_first=True)

X = df_clean.drop(columns='ModifiedDateDays')
y = df_clean['ModifiedDateDays']

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("R2 Score:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))


R2 Score: 0.7013708114407509
RMSE: 171.4463192754163
